In [2]:
import sacct
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
import dask.dataframe as dd
from random import randint
import numpy as np
from dask.array import from_array as fa

sourceFile = 'shortData.csv'  # Source of sacct data if using offline mode.

from dask_jobqueue import SLURMCluster
from dask.distributed import Client

cluster = SLURMCluster() #add arguments if want a cluster outside of the set up defult
cluster = SLURMCluster(cores=24, processes=6, walltime="3:00:00", queue="compute", memory= "120GB")

/home/dougfe/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43435 instead
  warnings.warn(


In [3]:
cluster.scale(jobs=2)

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://10.22.253.175:44624 Dashboard: http://10.22.253.175:43435/status,Cluster Workers: 12 Cores: 48 Memory: 240.00 GB


In [5]:
df = dd.read_csv(sourceFile)
#df = dd.read_parquet(sourceFile)  # For if you're smart and use parquet files instead of csvs

In [6]:
pd.set_option('display.max.columns', None)
pd.set_option('display.width', 150)
print(df.shape,'\n\n')
print(df.head(),'\n\n')
df.describe()

(Delayed('int-2e7b7c74-4cd8-4ca6-91f3-871baba20d75'), 76) 


   Unnamed: 0  JobIDRaw   step           JobID  UID  GID  AssocID Cluster                 JobName  User  Group Account  Reservation  ReservationId  \
0           0  35767092  batch  35767092.batch  NaN  NaN    24786   comet                   batch   NaN    NaN  mit178          NaN            NaN   
1           1  35767116  batch  35767116.batch  NaN  NaN    24786   comet                   batch   NaN    NaN  mit178          NaN            NaN   
2           2  35790946      0      35790946.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN            NaN   
3           3  35790948      0      35790948.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN            NaN   
4           4  35790951      0      35790951.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN            NaN   

   Partition  QOS  QOSRAW  NNODES  NTA

,Unnamed: 0,JobIDRaw,UID,GID,AssocID,User,Group,Reservation,ReservationId,Partition,QOS,QOSRAW,NNODES,NTASKS,NCPUS,AllocCPUS,ReqCPUS,Timelimit,Priority,DerivedExitCode,Time,MinCPUTask,ResvCPURaw,CPUTimeRaw,MaxDiskReadTask,MaxDiskWriteTask,MaxPagesTask,MaxRSSTask,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Comment
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [24]:
# Get info on single account usage
queryDF = df
#print(userDF[['Account', 'JobIDRaw', 'JobName', 'State', 'ExitCode', 'Start', 'End']])

In [17]:
# Get info on multiple account usage

accountList = ['mit178', 'dku133', 'cla176']

queryDF = df.loc[df['Account'].isin(accountList)]
#print(userDF[['Account', 'JobIDRaw', 'JobName', 'State', 'ExitCode', 'Start']].compute())

In [27]:
for orig in queryDF['MaxRSS']:
    if (type(orig) != int) and (type(orig) != float):
        if (orig[-1] == "K"):
            num = float(orig[0 : -1])
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig[-1] == "M"):
            num = float(orig[0 : -1]) * 1024
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig[-1] == "G"):
            num = float(orig[0 : -1]) * 1024 * 1024
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig == ""):
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, 0.0)

In [26]:
len(queryDF.query('MaxRSS > 5000000 '))

ValueError: Metadata inference failed in `query`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("'>' not supported between instances of 'str' and 'int'")

Traceback:
---------
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask/dataframe/utils.py", line 180, in raise_on_meta_error
    yield
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask/dataframe/core.py", line 5508, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask/utils.py", line 901, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py", line 3231, in query
  

In [ ]:
# Graphing memory usage (experimental)
# TODO: Only works if caolinn manages to replace maxrss column with raw kb data!
queryDF.set_index('MaxRSS')
sns.histplot(data=queryDF, x = 'MaxRSS')

In [10]:
# Display query dataframe preview
queryDF.head(50)

,Unnamed: 0,JobIDRaw,step,JobID,UID,GID,AssocID,Cluster,JobName,User,Group,Account,Reservation,ReservationId,Partition,QOS,QOSRAW,NNODES,NTASKS,NCPUS,AllocCPUS,ReqCPUS,ReqCPUFreq,ReqMem,ReqGRES,AllocGRES,Timelimit,Priority,State,ExitCode,DerivedExitCode,Submit,Eligible,Start,End,Time,Elapsed,Reserved,Suspended,AveCPU,MinCPU,MinCPUNode,MinCPUTask,ResvCPU,ResvCPURaw,TotalCPU,SystemCPU,UserCPU,CPUTime,CPUTimeRaw,AveCPUFreq,AveDiskRead,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,AveDiskWrite,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,AvePages,MaxPages,MaxPagesNode,MaxPagesTask,AveRss,MaxRSS,MaxRSSNode,MaxRSSTask,AveVMSize,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Layout,Comment,NodeList
365,35185.0,33823828,batch,33823828.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,5Gc,NaN,NaN,NaN,NaN,CANCELLED,0:15,NaN,2020-06-01T13:31:20,2020-06-01T13:31:20,2020-06-01T13:31:20,2020-06-01T14:06:33,NaN,00:35:13,INVALID,00:00:00,06:15:38,06:15:38,comet-02-57,0.0,INVALID,NaN,06:15:38,00:27.012,06:15:11,07:02:36,25356,15.90M,5529M,5529M,comet-02-57,0.0,2973M,2973M,comet-02-57,0.0,223K,223K,comet-02-57,0.0,1369564K,1369564K,comet-02-57,0.0,3248308K,3553124K,comet-02-57,0.0,0.0,0.0,Unknown,NaN,comet-02-57
4582,35273.0,33823902,batch,33823902.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,5Gc,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T13:41:46,2020-06-01T13:41:46,2020-06-01T13:41:46,2020-06-02T01:00:38,NaN,11:18:52,INVALID,00:00:00,5-03:56:15,5-03:56:15,comet-01-09,0.0,INVALID,NaN,5-03:58:17,18:46.836,5-03:39:30,5-15:46:24,488784,427K,125961M,125961M,comet-01-09,0.0,77733M,77733M,comet-01-09,0.0,223K,223K,comet-01-09,0.0,1663044K,1771916K,comet-01-09,0.0,3433548K,3.63G,comet-01-09,0.0,0.0,0.0,Unknown,NaN,comet-01-09
4711,88515.0,33866415,batch,33866415.batch,NaN,NaN,16992,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,0,4Gn,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-03T07:18:02,2020-06-03T07:18:02,2020-06-03T07:18:02,2020-06-03T07:24:53,NaN,00:06:51,INVALID,00:00:00,00:05:19,00:05:19,comet-01-06,0.0,INVALID,NaN,05:35.830,00:05.397,05:30.433,00:06:51,411,2.87G,3626M,3626M,comet-01-06,0.0,0.00M,0.00M,comet-01-06,0.0,38K,38K,comet-01-06,0.0,65100K,65108K,comet-01-06,0.0,543652K,543652K,comet-01-06,0.0,0.0,0.0,Unknown,NaN,comet-01-06
5897,31037.0,33820892,batch,33820892.batch,NaN,NaN,24786,comet,batch,NaN,NaN,mit178,NaN,NaN,NaN,NaN,NaN,1,1,24,24,24,0,4000Mc,NaN,NaN,NaN,NaN,CANCELLED,0:15,NaN,2020-06-01T11:47:01,2020-06-01T11:47:01,2020-06-01T11:47:01,2020-06-01T13:12:53,NaN,01:25:52,INVALID,00:00:00,1-01:09:52,1-01:09:52,comet-21-35,0.0,INVALID,NaN,1-01:09:52,11:24.007,1-00:58:28,1-10:20:48,123648,6.67M,80984M,80984M,comet-21-35,0.0,24888M,24888M,comet-21-35,0.0,237K,237K,comet-21-35,0.0,5223132K,5947592K,comet-21-35,0.0,9092700K,12722144K,comet-21-35,0.0,0.0,0.0,Unknown,NaN,comet-21-35
6552,58458.0,33841060,batch,33841060.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,5Gc,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-02T07:21:01,2020-06-02T07:21:01,2020-06-02T07:21:01,2020-06-02T08:56:41,NaN,01:35:40,INVALID,00:00:00,17:51:03,17:51:03,comet-07-10,0.0,INVALID,NaN,17:51:39,03:24.785,17:48:14,19:08:00,68880,16.57M,22040M,22040M,comet-07-10,0.0,17291M,17291M,comet-07-10,0.0,24K,24K,comet-07-10,0.0,988688K,988688K,comet-07-10,0.0,2864344K,3098516K,comet-07-10,0.0,0.0,0.0,Unknown,NaN,comet-07-10
7116,32904.0,33822148,batch,33822148.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,5Gc,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T12:29:14,2020-06-01T12:29:14,2020-06-01T12:29:14,2020-06-01T12:55:15,NaN,00:26:01,INVALID,00:00:00,04:55:42,04:55:42,comet-10-46,0.0,INVALID,NaN,05:01:04,01:05.466,04:59:58,05:12:12,18732,57.57M,16496M,16496M,comet-10-46,0.0,5210M,5210M,comet-10-46,0.0,171K,171K,comet-10-46,0.0,568048K,570488K,comet-10-46,0.0,2313004K,2607952K,comet-10-46,0.0,0.0,0.0,Unknown,Na

In [14]:
queryDF.compute().describe()

,Unnamed: 0
count,6112.000000
mean,47662.154450
std,26994.630331
min,1908.000000
25%,32910.000000
50%,36622.000000
75%,58505.000000
max,172581.000000


In [ ]:
# Query save system: If desired, users can save their current query as a parquet file for use later

saveName = 'sacctQuery'  # Doesn't need to include the .parqet or .csv extension
queryDF.to_parquet(saveName+'.parquet')